<a href="https://colab.research.google.com/github/chaudharynidhi/Image_Classification/blob/master/Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
import os
import PIL
import PIL.Image
import pathlib
import shutil
import autokeras as ak


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -uq "/content/drive/MyDrive/data_intel/archive.zip" -d "/content/drive/MyDrive/data_intel/"

In [ ]:
train_data = "/content/drive/MyDrive/data_intel/seg_train/seg_train"
test_data = "/content/drive/MyDrive/data_intel/seg_test/seg_test"
pred_data = "/content/drive/MyDrive/data_intel/seg_pred/seg_pred"

In [ ]:
train_data

'/content/drive/MyDrive/data_intel/seg_train/seg_train'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, 
                                   horizontal_flip=True)

train_set = train_datagen.flow_from_directory(train_data, target_size = (224,224), 
                                              batch_size=4, class_mode = 'sparse')

Found 14034 images belonging to 6 classes.


In [ ]:
import tensorflow.keras.preprocessing.image

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, 
                                  horizontal_flip=True)

test_set = test_datagen.flow_from_directory(test_data, target_size = (224,224), 
                                            batch_size=4, class_mode = 'sparse')

Found 3000 images belonging to 6 classes.


In [ ]:
type(train_set)

tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [ ]:
len(test_set.class_indices)

6

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.InputLayer(input_shape=(224,224,3)),
                             tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2,2), activation='relu'),
                             tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2,2), activation='relu'),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6)
])

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_set, 
          steps_per_epoch=len(train_set)//batch_size, 
          validation_data=test_set, 
          epochs=2)

Epoch 1/2
109/109 [==============================] - 152s 1s/step - loss: 10.9499 - accuracy: 0.1621 - val_loss: 11.3794 - val_accuracy: 0.1697
Epoch 2/2
109/109 [==============================] - 145s 1s/step - loss: 11.4928 - accuracy: 0.1628 - val_loss: 11.3794 - val_accuracy: 0.1697


In [ ]:
batch_size=4


In [ ]:
final_image = tf.keras.applications.xception.preprocess_input
final_image

<function tensorflow.python.keras.applications.xception.preprocess_input>

In [ ]:
base_model = tf.keras.applications.Xception(input_shape=(224,224,3), include_top=False, 
                                            classifier_activation='sigmoid', classes=len(train_set.class_indices))

In [ ]:
image_batch, label_batch = next(iter(train_set))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(4, 7, 7, 2048)


In [ ]:
base_model.trainable=False

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(4, 2048)


In [ ]:
prediction_layer = tf.keras.layers.Dense(6, activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(4, 6)


In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = final_image(inputs)
x = base_model(x, training = False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.math.truediv_6 (TFOpLambd (None, 224, 224, 3)       0         
_________________________________________________________________
tf.math.subtract_6 (TFOpLamb (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 1229

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.math.truediv_6 (TFOpLambd (None, 224, 224, 3)       0         
_________________________________________________________________
tf.math.subtract_6 (TFOpLamb (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 1229

In [ ]:
history = model.fit(train_set,
                    epochs=20,
                    validation_data=test_set)

Epoch 1/20
3509/3509 [==============================] - 3908s 1s/step - loss: 1.7914 - accuracy: 0.1896 - val_loss: 1.7656 - val_accuracy: 0.2053
Epoch 2/20
 151/3509 [>.............................] - ETA: 51:18 - loss: 1.7685 - accuracy: 0.2292

In [ ]:
model.trainable_variables

[<tf.Variable 'dense_3/kernel:0' shape=(2048, 6) dtype=float32, numpy=
 array([[-0.02836046,  0.01168105, -0.04344233,  0.03095152, -0.05373045,
         -0.00845481],
        [-0.02617168, -0.04137754, -0.05032866,  0.0356075 , -0.04183048,
         -0.00608218],
        [-0.04196942, -0.01333203, -0.03917311,  0.03934994, -0.04644724,
          0.00658315],
        ...,
        [ 0.03925906, -0.04055894,  0.04511383,  0.00111177, -0.0352831 ,
         -0.04160343],
        [ 0.01716872,  0.00101524,  0.03354238, -0.01607356,  0.00758892,
          0.00993709],
        [-0.01567215, -0.02462473, -0.05081228,  0.01305764, -0.03693835,
         -0.03612506]], dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(6,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [ ]:
model.evaluate(test_set)

ValueError: ignored

In [ ]:
test_set[0]

AttributeError: ignored

In [ ]:
image_batch, label_batch = next(iter(test_set))

In [ ]:
image_batch

array([[[[0.00639634, 0.39517412, 0.6644257 ],
         [0.00221339, 0.40050524, 0.65932876],
         [0.00649252, 0.4024152 , 0.6567104 ],
         ...,
         [0.40372664, 0.63490427, 0.7997076 ],
         [0.4039216 , 0.63529414, 0.8000001 ],
         [0.39210218, 0.63529414, 0.80295485]],

        [[0.00971728, 0.39441577, 0.6670197 ],
         [0.00553433, 0.40069607, 0.6611324 ],
         [0.00781078, 0.40304768, 0.6580666 ],
         ...,
         [0.4097019 , 0.6409529 , 0.80553615],
         [0.40980005, 0.64117265, 0.8058785 ],
         [0.4023688 , 0.6367375 , 0.8021924 ]],

        [[0.01201435, 0.38982162, 0.667166  ],
         [0.00784314, 0.39607847, 0.65882355],
         [0.00784314, 0.39897862, 0.65689015],
         ...,
         [0.4140176 , 0.6430007 , 0.802811  ],
         [0.41410333, 0.6431373 , 0.803166  ],
         [0.4081819 , 0.6372276 , 0.7972798 ]],

        ...,

        [[0.01450393, 0.07191985, 0.15440114],
         [0.01684628, 0.07010052, 0.15249962]

In [ ]:
final_image(test_set)

TypeError: ignored

In [ ]:
label_batch

array([2., 4., 0., 4.], dtype=float32)